In [2]:
import pandas as pd
import openai
import json
import os

openai.api_key = os.environ["OPENAI_API_KEY"]
train_df = pd.read_csv("/home/clay/research/kaggle/kaggle_llm/data/kaggle-llm-science-exam/train.csv", index_col=0)

In [4]:
def create_prompt(row):
    return f'''
{row.prompt}
A) {row.A}
B) {row.B}
C) {row.C}
D) {row.D}
E) {row.E}
Select the 3 best answers for that question then rank them from best to worst
'''


def Answer_question_LLM(prompt):
    messages = [{
        "role": "system", 
        "content": f'You are an AI designed to help me answer questions'
    }]
    message=f"{prompt} "
    messages.append({"role": "user", "content": message})
    response = openai.ChatCompletion.create(
#             model="gpt-4-0613",
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=[
            {
                "name": "Answer_question",
                "description": "Answer a given multiple choice question",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "Answer": {
                            "type": "string",
                            "description": "The 3 best answers for that question then rank them from best to worst. Example : 'A B C'",
                        },
                    },
                    "required": ["Answer"],
                },
            }
        ],
        function_call={"name": "Answer_question"},
    )

    assistant_msg = response['choices'][0]['message']
    response_options = assistant_msg.to_dict()['function_call']['arguments']
    options = json.loads(response_options)
    return options["Answer"]

    
submission_df = pd.DataFrame(columns=["id", "prediction"])
for index, row in train_df.head(1).iterrows():
    prompt = create_prompt(row)
    prediction = Answer_question_LLM(prompt)
    submission_df = submission_df.append({"id": index, "prediction": prediction}, ignore_index=True)

/tmp/ipykernel_206746/2629185306.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission_df = submission_df.append({"id": index, "prediction": prediction}, ignore_index=True)
/tmp/ipykernel_206746/2629185306.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission_df = submission_df.append({"id": index, "prediction": prediction}, ignore_index=True)
/tmp/ipykernel_206746/2629185306.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission_df = submission_df.append({"id": index, "prediction": prediction}, ignore_index=True)
/tmp/ipykernel_206746/2629185306.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submissio

,id,prediction
0,0,A D E
1,1,A B C
2,2,A B C
3,3,A C D
4,4,B A E


In [6]:
response

NameError: name 'response' is not defined

In [ ]:
# Save the submission dataframe to a csv file
submission_df.to_csv('GPT-4-train.csv', index=False)